<font size="+3.8">Scanpy single-cell pre-processing</font>  
<font size="+1.5"></font>  

Aim: Preprocess annotated mouse brain single-cell data from Zeisel et al 2018 (160k cells stored as loom file)

In [ ]:
from datetime import date
date.today().strftime('%d/%m/%Y')

In [ ]:
import os
os.getlogin()

In [ ]:
import sys
import platform, fnmatch 

In [ ]:
import anndata
import scanpy as sc
import scipy as sci
#sc.logging.print_versions()

In [ ]:
sc.settings.verbosity = 3

In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
import seaborn as sb
import datetime
now = datetime.datetime.now()
today = now.strftime("%Y%m%d")

In [ ]:
from matplotlib.pyplot import rc_context

In [ ]:
import utils

In [ ]:
os.environ['CONDA_DEFAULT_ENV'] # conda env

In [ ]:
platform.platform()

In [ ]:
main_dir='/run/user/1000/gvfs/smb-share:server=138.245.4.35,share=bd-dichgans/SF' # Linux
main_dir='\\\isdsynnas.srv.med.uni-muenchen.de\BD-Dichgans\SF' # Win
main_dir='/Volumes/BD-Dichgans/SF' # Mac

In [ ]:
dataset_name = "Zeisel2018"
organism = "Mouse"

# Load + format data

## Annotated by authors

Partly adapted from https://github.com/theislab/scib-reproducibility/blob/main/notebooks/data_preprocessing/mouse_brain/01_collect_mouse_brain_studies.ipynb

Data downloaded from: http://mousebrain.org/adolescent/downloads.html

In [ ]:
adata_zeisel = sc.read_loom(os.path.join(main_dir, "P06_vasc_scRNAseq", "Zeisel2018", "l5_all.loom"))

In [ ]:
adata_zeisel.obs_names_make_unique()
adata_zeisel.var_names_make_unique()

In [ ]:
adata_zeisel

In [ ]:
list(adata_zeisel.obs.columns)

In [ ]:
pd.value_counts(adata_zeisel.obs['Class'])

In [ ]:
pd.value_counts(adata_zeisel.obs['Taxonomy_group'])

In [ ]:
adata_zeisel.obs['Region'].astype('category').cat.categories

In [ ]:
pd.value_counts(adata_zeisel.obs['Age'])

In [ ]:
pd.value_counts(adata_zeisel.obs['Subclass'])

Assign regions to the already present brain structures:
* Amygdala - Amygdala (AMY)
* CA1 - Hippocampus (HC)
* Ctx1, Ctx1.5, Ctx2, Ctx3 - Cortex (CTX)
* DRG - Dorsal root ganglia (**to exclude**)
* DentGyr - Dentate Gyrus (to Hippocampus)
* ENS - Enteric nervous system (**to exclude**)
* HC - Hippocampus
* Hypoth - Hypothalamus (HTH)
* MBd, MBv - midbrain dorsal, ventral (merge to midbrain, MB)
* Medulla - MD
* OB - olfactory bulb
* Pons - PO 
* SC - spinal cord  (**to exclude**)
* SScortex - secondary somatosensensory cortex - Cortex (CTX)
* StriatDor, StriatVent - Striatum (STR)
* Sympath - Sympathetic ganglion (**to exclude**)
* Thal - Thalamus (TH)

In [ ]:
adata_zeisel.obs['Tissue'].astype('category').cat.categories

In [ ]:
region_dict = dict({'Amygd': 'AMY', 
                    'CA1' : 'HC', 
                    'Ctx1': 'CTX', 
                    'Ctx1.5': 'CTX', 
                    'Ctx2': 'CTX', 
                    'Ctx3': 'CTX', 
                    #'DRG',
                    'DentGyr' : 'HC', 
                    #'ENS', 
                    'Hypoth' : 'HTH', 
                    'MBd' : 'MB', 
                    'MBv' : 'MB', 
                    'Medulla' : 'MD', 
                    'Pons' : 'PO',
                    'SScortex' : 'CTX', 
                    'StriatDor' : 'STR', 
                    'StriatVent': 'STR',
                    #'Sympath', 
                    'Thal' : 'TH'})

df = pd.DataFrame.from_dict(region_dict, orient='index')

In [ ]:
adata_zeisel.obs['region'] = adata_zeisel.obs['Tissue'].astype('category').cat.add_categories(np.unique(
        ['AMY', 'CTX', 'HTH', 'MB', 'MD', 'PO', 'STR', 'TH']))

for idx in enumerate(df[0]):
    adata_zeisel.obs['region'].loc[adata_zeisel.obs['region']==df[0].index[idx[0]]] = idx[1]
    
adata_zeisel.obs['region'] = adata_zeisel.obs['region'].astype('category').cat.remove_unused_categories()

Exclude `DRG`, `ENS` and `Sympath` cells.  
= all peripheral NS cells

In [ ]:
adata_zeisel = adata_zeisel[np.invert(np.in1d(adata_zeisel.obs['region'], ['DRG', 'ENS', 'Sympath','SC']))].copy() # remove SC because only interested in brain

In [ ]:
adata_zeisel = adata_zeisel[np.invert(np.in1d(adata_zeisel.obs['Taxonomy_group'], ['Spinal cord excitatory neurons', 'Spinal cord inhibitory neurons']))].copy() # remove SC neurons

In [ ]:
adata_zeisel.obs['region'].value_counts()

In [ ]:
pd.value_counts(adata_zeisel.obs['Subclass'])

In [ ]:
pd.value_counts(adata_zeisel.obs['Age'])

In [ ]:
pd.value_counts(adata_zeisel.obs['Sex'])

In [ ]:
pd.value_counts(adata_zeisel.obs['DonorID'])

In [ ]:
pd.crosstab(adata_zeisel.obs['Taxonomy_group'],adata_zeisel.obs['Subclass'])

In [ ]:
adata_zeisel

In [ ]:
del adata_zeisel.obs['Region']

Scib re-grouped cell types as in the following. I did re-grouping myself, based on cell types of interest.

<s>Let us use the `Taxonomy_group` of `adata_zeisel` as a new variable `cell_type`. Then, *oligos* are renamed to *oligodendrocytes*, *vascular* becomes *endothelial cell*. The *Immune* cells are a mixture of *macrophages* and *microglia*. We have to check the *Peripheral glia* and *Ependymal*, which have no correspondence in the other datasets. However, microglia are apparently missing in the dataset.</s>

In [ ]:
sc.pl.violin(adata_zeisel, ['Lyz2', 'Ctss', 'Cd74'], groupby='Taxonomy_group')

**Conclusion** Peripheral Glia have most overlap with brain pericytes and we keep the ependymal cells as extra category.

In [ ]:
adata_zeisel.obs['Taxonomy_group'].cat.categories

In [ ]:
# # from scib code (not used)
# adata_zeisel.obs['cell_type'] = adata_zeisel.obs['Taxonomy_group'].cat.add_categories(['astrocyte', 
#         'brain pericyte', 'endothelial cell','ependymal cell','neuron',
#        'macrophage', 'microglial cell', 'oligodendrocyte',
#                      'oligodendrocyte precursor cell',
#                          'olfactory ensheathing cell', ])

# adata_zeisel.obs['cell_type'][np.in1d(adata_zeisel.obs['Taxonomy_group'], 
#                                      ['Astrocytes','Subventricular zone radial glia-like cells',
#                                      'Dentate gyrus radial glia-like cells'])] =  'astrocyte'
# adata_zeisel.obs['cell_type'][np.in1d(adata_zeisel.obs['Taxonomy_group'], 
#                                      ['Ependymal cells', 
#                                       'Subcommissural organ hypendymal cells'])] = 'ependymal cell'
         
# adata_zeisel.obs['cell_type'][np.in1d(adata_zeisel.obs['Taxonomy_group'], 
#                                      ['Perivascular macrophages'])] = 'macrophage'
# adata_zeisel.obs['cell_type'][np.in1d(adata_zeisel.obs['Taxonomy_group'], 
#                                      ['Microglia'])] = 'microglial cell'
           
# adata_zeisel.obs['cell_type'][np.in1d(adata_zeisel.obs['Taxonomy_group'], 
#                                      ['Telencephalon inhibitory interneurons',
#        'Telencephalon projecting excitatory neurons',
#        'Telencephalon projecting inhibitory neurons', 
#                                       'Di- and mesencephalon excitatory neurons',
#        'Di- and mesencephalon inhibitory neurons',
#                                       'Cerebellum neurons',
#        'Cholinergic and monoaminergic neurons',
#                                       'Dentate gyrus granule neurons',
#                                       'Non-glutamatergic neuroblasts',
#                                       'Glutamatergic neuroblasts', 'Hindbrain neurons',
#                                       'Spinal cord excitatory neurons',
#                                       'Olfactory inhibitory neurons',
#                                       'Peptidergic neurons',
#        'Spinal cord inhibitory neurons'])] = 'neuron'

# adata_zeisel.obs['cell_type'][np.in1d(adata_zeisel.obs['Taxonomy_group'], 
#                                      ['Oligodendrocytes'])] = 'oligodendrocyte'

# adata_zeisel.obs['cell_type'][np.in1d(adata_zeisel.obs['Taxonomy_group'], 
#                                      ['Oligodendrocyte precursor cells'])] = 'oligodendrocyte precursor cell'

# adata_zeisel.obs['cell_type'][np.in1d(adata_zeisel.obs['Taxonomy_group'], 
#                                      ['Olfactory ensheathing cells'])] = 'olfactory ensheathing cell'

# adata_zeisel.obs['cell_type'][np.in1d(adata_zeisel.obs['Taxonomy_group'], 
#                                      ['Pericytes'])] = 'brain pericyte'

# adata_zeisel.obs['cell_type'][np.in1d(adata_zeisel.obs['Taxonomy_group'], 
#                                      ['Vascular and leptomeningeal cells', 'Vascular endothelial cells',
#        'Vascular smooth muscle cells', 'Choroid epithelial cells'])] = 'endothelial cell'

# adata_zeisel.obs['cell_type'] = adata_zeisel.obs['cell_type'].cat.remove_unused_categories()


In [ ]:
# updated own (used)
adata_zeisel.obs['cell_type'] = adata_zeisel.obs['Taxonomy_group'].cat.add_categories(
        ['astrocytes','pericytes', 'Endothelial cells','Ependymal cell','Neurons','Neuroblasts',
        'Microglia/Macrophages', 'Oligos','SMCs', 'OPCs','olfactory ensheathing cells', 'Leptomeningeal cells']) # cannot be same name as "Taxonomy_group"

In [ ]:
list(adata_zeisel.obs['Taxonomy_group'].unique())

In [ ]:
adata_zeisel.obs['cell_type'][np.in1d(adata_zeisel.obs['Taxonomy_group'], 
                                     ['Astrocytes','Subventricular zone radial glia-like cells',
                                     'Dentate gyrus radial glia-like cells'])] =  'Astrocytes'

adata_zeisel.obs['cell_type'][np.in1d(adata_zeisel.obs['Taxonomy_group'], 
                                     ['Ependymal cells','Subcommissural organ hypendymal cells', 'Choroid epithelial cells'])] = 'Ependymal cells'
         
adata_zeisel.obs['cell_type'][np.in1d(adata_zeisel.obs['Taxonomy_group'], 
                                     ['Microglia','Perivascular macrophages'])] = 'Microglia/Macrophages'

adata_zeisel.obs['cell_type'][np.in1d(adata_zeisel.obs['Taxonomy_group'], 
                                     [ 'Cholinergic and monoaminergic neurons','Telencephalon projecting excitatory neurons','Telencephalon inhibitory interneurons',
                                      'Olfactory inhibitory neurons','Peptidergic neurons','Di- and mesencephalon excitatory neurons','Hindbrain neurons',
                                      'Telencephalon projecting inhibitory neurons','Dentate gyrus granule neurons','Cerebellum neurons',
                                      'Di- and mesencephalon inhibitory neurons'])] = 'Neurons'

adata_zeisel.obs['cell_type'][np.in1d(adata_zeisel.obs['Taxonomy_group'], 
                                     ['Glutamatergic neuroblasts',
                                      'Non-glutamatergic neuroblasts'])] = 'Neuroblasts'

adata_zeisel.obs['cell_type'][np.in1d(adata_zeisel.obs['Taxonomy_group'], 
                                     ['Oligodendrocytes'])] = 'Oligos'

adata_zeisel.obs['cell_type'][np.in1d(adata_zeisel.obs['Taxonomy_group'], 
                                     ['Oligodendrocyte precursor cells'])] = 'OPCs'

adata_zeisel.obs['cell_type'][np.in1d(adata_zeisel.obs['Taxonomy_group'], 
                                     ['Olfactory ensheathing cells'])] = 'Olfactory ensheathing cells'

adata_zeisel.obs['cell_type'][np.in1d(adata_zeisel.obs['Taxonomy_group'], 
                                     ['Pericytes'])] = 'Pericytes'

adata_zeisel.obs['cell_type'][np.in1d(adata_zeisel.obs['Taxonomy_group'], 
                                     ['Vascular endothelial cells'])] = 'Endothelial cells'

adata_zeisel.obs['cell_type'][np.in1d(adata_zeisel.obs['Taxonomy_group'], 
                                     ['Vascular smooth muscle cells'])] = 'SMCs'

adata_zeisel.obs['cell_type'][np.in1d(adata_zeisel.obs['Taxonomy_group'], 
                                     ['Vascular and leptomeningeal cells'])] = 'Leptomeningeal cells'

adata_zeisel.obs['cell_type'] = adata_zeisel.obs['cell_type'].cat.remove_unused_categories()


In [ ]:
adata_zeisel.obs['cell_type'].value_counts()

In [ ]:
adata_zeisel.obs['study'] = 'Zeisel'

In [ ]:
##Keep tissue, subclass, age and region.
#adata_zeisel.obs['Region']
#adata_zeisel.obs['Age']
#adata_zeisel.obs['Subclass']
#adata_zeisel.obs['Tissue']

In [ ]:
del adata_zeisel.obs['AnalysisPool']
del adata_zeisel.obs['AnalysisProject']
del adata_zeisel.obs['Bucket']
del adata_zeisel.obs['CellConc']
del adata_zeisel.obs['Cell_Conc']
del adata_zeisel.obs['ChipID']
del adata_zeisel.obs['Class']
del adata_zeisel.obs['ClassProbability_Astrocyte']
del adata_zeisel.obs['ClassProbability_Astrocyte,Immune']
del adata_zeisel.obs['ClassProbability_Astrocyte,Neurons']
del adata_zeisel.obs['ClassProbability_Astrocyte,Oligos']
del adata_zeisel.obs['ClassProbability_Astrocyte,Vascular']
del adata_zeisel.obs['ClassProbability_Bergmann-glia']
del adata_zeisel.obs['ClassProbability_Blood']
del adata_zeisel.obs['ClassProbability_Blood,Vascular']
del adata_zeisel.obs['ClassProbability_Enteric-glia']
del adata_zeisel.obs['ClassProbability_Enteric-glia,Cycling']
del adata_zeisel.obs['ClassProbability_Ependymal']
del adata_zeisel.obs['ClassProbability_Ex-Neurons']
del adata_zeisel.obs['ClassProbability_Ex-Vascular']
del adata_zeisel.obs['ClassProbability_Immune']
del adata_zeisel.obs['ClassProbability_Immune,Neurons']
del adata_zeisel.obs['ClassProbability_Immune,Oligos']
del adata_zeisel.obs['ClassProbability_Neurons']
del adata_zeisel.obs['ClassProbability_Neurons,Cycling']
del adata_zeisel.obs['ClassProbability_Neurons,Oligos']
del adata_zeisel.obs['ClassProbability_Neurons,Satellite-glia']
del adata_zeisel.obs['ClassProbability_Neurons,Vascular']
del adata_zeisel.obs['ClassProbability_OEC']
del adata_zeisel.obs['ClassProbability_Oligos']
del adata_zeisel.obs['ClassProbability_Oligos,Cycling']
del adata_zeisel.obs['ClassProbability_Oligos,Vascular']
del adata_zeisel.obs['ClassProbability_Satellite-glia']
del adata_zeisel.obs['ClassProbability_Satellite-glia,Cycling']
del adata_zeisel.obs['ClassProbability_Satellite-glia,Schwann']
del adata_zeisel.obs['ClassProbability_Schwann']
del adata_zeisel.obs['ClassProbability_Ttr'] 
del adata_zeisel.obs['ClassProbability_Vascular']
del adata_zeisel.obs['ClusterName']
del adata_zeisel.obs['Clusters']
del adata_zeisel.obs['Comment']
del adata_zeisel.obs['Comments']
del adata_zeisel.obs['DateCaptured']
del adata_zeisel.obs['Date_Captured']
del adata_zeisel.obs['Description']
del adata_zeisel.obs['Developmental_compartment']
del adata_zeisel.obs['DonorID']
del adata_zeisel.obs['Estimated Number of Cells']
del adata_zeisel.obs['Flowcell']
del adata_zeisel.obs['Fraction Reads in Cells']
del adata_zeisel.obs['Label']
del adata_zeisel.obs['LeafOrder']
del adata_zeisel.obs['Location_based_on']
del adata_zeisel.obs['Mean Reads per Cell']
del adata_zeisel.obs['Median Genes per Cell']
del adata_zeisel.obs['Median UMI Counts per Cell']
del adata_zeisel.obs['MitoRiboRatio']
del adata_zeisel.obs['NGI_PlateWell']
del adata_zeisel.obs['Neurotransmitter']
del adata_zeisel.obs['NumPooledAnimals']
del adata_zeisel.obs['Num_Pooled_Animals']
del adata_zeisel.obs['Number of Reads']
del adata_zeisel.obs['OriginalClusters']
del adata_zeisel.obs['Outliers']
del adata_zeisel.obs['PCRCycles']
del adata_zeisel.obs['PCR_Cycles']
del adata_zeisel.obs['PassedQC']
del adata_zeisel.obs['PlugDate']
del adata_zeisel.obs['Plug_Date']
del adata_zeisel.obs['Probable_location']
del adata_zeisel.obs['Project']
del adata_zeisel.obs['Q30 Bases in Barcode']
del adata_zeisel.obs['Q30 Bases in RNA Read']
del adata_zeisel.obs['Q30 Bases in Sample Index']
del adata_zeisel.obs['Q30 Bases in UMI']
del adata_zeisel.obs['Reads Mapped Confidently to Exonic Regions']

In [ ]:
del adata_zeisel.obs['Reads Mapped Confidently to Intergenic Regions']
del adata_zeisel.obs['Reads Mapped Confidently to Intronic Regions']
del adata_zeisel.obs['Reads Mapped Confidently to Transcriptome']
del adata_zeisel.obs['SampleID']
del adata_zeisel.obs['SampleIndex']
del adata_zeisel.obs['SampleOK']
del adata_zeisel.obs['Sample_Index']
del adata_zeisel.obs['SeqComment']
del adata_zeisel.obs['SeqLibDate']
del adata_zeisel.obs['SeqLibOk']
del adata_zeisel.obs['Seq_Comment']
del adata_zeisel.obs['Seq_Lib_Date']
del adata_zeisel.obs['Seq_Lib_Ok']
del adata_zeisel.obs['Sequencing Saturation']
del adata_zeisel.obs['Serial_Number']
del adata_zeisel.obs['Sex']
del adata_zeisel.obs['Species']
del adata_zeisel.obs['Strain']
del adata_zeisel.obs['TargetNumCells']
del adata_zeisel.obs['Target_Num_Cells']
del adata_zeisel.obs['TaxonomyRank1']
del adata_zeisel.obs['TaxonomyRank2']
del adata_zeisel.obs['TaxonomyRank3']
del adata_zeisel.obs['TaxonomyRank4']
del adata_zeisel.obs['TaxonomySymbol']
del adata_zeisel.obs['TimepointPool']
del adata_zeisel.obs['Total Genes Detected']
del adata_zeisel.obs['Transcriptome']
del adata_zeisel.obs['Valid Barcodes']
del adata_zeisel.obs['_KMeans_10']
del adata_zeisel.obs['_LogCV']
del adata_zeisel.obs['_LogMean']
del adata_zeisel.obs['_NGenes']
del adata_zeisel.obs['_PC1']
del adata_zeisel.obs['_PC2']
del adata_zeisel.obs['_Total']
del adata_zeisel.obs['_Valid']
del adata_zeisel.obs['_X']
del adata_zeisel.obs['_Y']
del adata_zeisel.obs['_tSNE1']
del adata_zeisel.obs['_tSNE2']
del adata_zeisel.obs['cDNAConcNanogramPerMicroliter']
del adata_zeisel.obs['cDNALibOk']
del adata_zeisel.obs['cDNA_Lib_Ok']
del adata_zeisel.obs['ngperul_cDNA']

In [ ]:
del adata_zeisel.var['Accession']
del adata_zeisel.var['_LogCV']
del adata_zeisel.var['_LogMean']
del adata_zeisel.var['_Selected']
del adata_zeisel.var['_Total']
del adata_zeisel.var['_Valid']

In [ ]:
adata = adata_zeisel

In [ ]:
adata_zeisel

In [ ]:
adata_zeisel

In [ ]:
sc.write(adata=adata_zeisel, filename=main_dir+'/P06_Foxf2_per_celltype/scRNAseq/'+date.today().strftime("%Y%m%d")+'_Zeisel2018.h5ad')

---

## Load previous work

In [ ]:
# load final file
#date_set='20230614'
#adata=sc.read_h5ad(main_dir+'/P06_Foxf2_per_celltype/scRNAseq/'+date_set+'_Zeisel2018.h5ad')
#adata.uns['log1p']['base'] = None

In [ ]:
# load final file
date_set='20231109'
adata=sc.read_h5ad(main_dir+'/P06_Foxf2_per_celltype/scRNAseq/20230228_zeisel_normalised_logarithmised_annotated.h5ad')
adata.uns['log1p']['base'] = None

In [ ]:
adata

# QC

Not required because already annotated

In [ ]:
# genes with highest fraction of counts per cell
sc.pl.highest_expr_genes(adata, n_top=20, )

# Normalisation, logarithmization

In [ ]:
adata.layers

In [ ]:
# show expression of 100 random genes (across all spots)
import random
import seaborn as sns
random_genes=random.sample(range(0, adata.X.shape[1]), 100)
adata_sub = adata[:,random_genes]
exp=pd.DataFrame(adata_sub.X.todense())
# plot
pl1=sns.displot(data=pd.melt(exp),x='value',height=4,hue='variable',kind="kde",warn_singular=False,legend=False,palette=list(np.repeat('#086da6',100)), lw=0.3) # genes with 0 expression are excluded
pl1.set(xlim=(-0.5, 7),ylim=(0,0.007));

In [ ]:
sns.set(rc={'figure.figsize':(4,4)})
sns.set_theme(style='white')
pl=sns.histplot(data=pd.melt(exp),x='value',binwidth=0.5,legend=True,palette=list(np.repeat('#086da6',100)))
pl.set(xlim=(0, 20),ylim=(0,1e6));

In [ ]:
adata.layers["counts"] = adata.X.copy() # save unnormalized raw RNA counts - retrieve via adata.X = adata.layers["counts"]

In [ ]:
sc.pp.normalize_total(adata, inplace=True) # Normalize each spot by total counts over all genes, so that every spot has the same total count after normalization.

In [ ]:
# show expression of 100 random genes (across all spots)
adata_sub = adata[:,random_genes]
exp=pd.DataFrame(adata_sub.X.todense())
# plot
pl=sns.displot(data=pd.melt(exp),x='value',height=4,hue='variable',kind="kde",warn_singular=False,legend=False,palette=list(np.repeat('#086da6',100)), lw=0.3)
pl.set(xlim=(-0.25, 3.5),ylim=(0,0.005))

In [ ]:
pl=sns.histplot(data=pd.melt(exp),x='value',binwidth=0.5,legend=True,palette=list(np.repeat('#086da6',100)))
pl.set(xlim=(0, 20),ylim=(0,1e6));

In [ ]:
sc.pp.log1p(adata) # X = log(X + 1)

In [ ]:
# show expression of 100 random genes (across all spots)
adata_sub = adata[:,random_genes]
exp=pd.DataFrame(adata_sub.X.todense())
# plot
pl=sns.displot(data=pd.melt(exp),x='value',height=4,hue='variable',kind="kde",warn_singular=False,legend=False,palette=list(np.repeat('#086da6',100)), lw=0.5) # genes with 0 expression are excluded
pl.set(xlim=(-0.25, 3.5),ylim=(0,0.005));

In [ ]:
pl=sns.histplot(data=pd.melt(exp),x='value',binwidth=0.5,legend=True,palette=list(np.repeat('#086da6',100)));
pl.set(xlim=(0, 20),ylim=(0,1e6));

In [ ]:
adata.layers["normalized"] = adata.X.copy() # save normalized + log-transformed (but unscaled) counts - retrieve via adata.X = adata.layers["normalized"]

In [ ]:
# Identify highly-variable genes
sc.pp.highly_variable_genes(adata)
sc.pl.highly_variable_genes(adata)

In [ ]:
adata

In [ ]:
adata.layers

# Dim Reduction

In [ ]:
# Run PCA
with rc_context({'figure.figsize': (8, 8)}):
    sc.tl.pca(adata, svd_solver='arpack')
    sc.pl.pca(adata, color='Foxf2')

In [ ]:
sc.pl.pca_variance_ratio(adata, log=True)

In [ ]:
sc.pp.neighbors(adata)

In [ ]:
# Run UMAP
sc.tl.umap(adata)

In [ ]:
list(adata.obs.columns)

In [ ]:
with rc_context({'figure.figsize': (9, 9)}):
    sc.pl.umap(adata, color=['cell_type', 'region'], wspace=0.3, size=2)

In [ ]:
plt.rcParams['figure.figsize'] = [11, 8] # set plot sizes
sc.pl.umap(adata, color=['cell_type'], legend_loc='on data', title='', legend_fontweight='normal', legend_fontoutline=3, legend_fontsize=14, size=3)

Note: UMAP does not separate cell types very well, nonetheless stick to pre-annotated cell types

# Cell annotation

Verify annotation from authors

Manual marker gene selection

|Vascular     |EC         |Pericytes|SMCs   |Fibroblasts|Oligos|OPCs         |Ependymal|Neurons    |immature/migrating Neurons|Astrocytes|Microglia|Immune (broad/hematopoetic)|Macrophages     |Macrophages/Microglia|Monocytes|Mononcytes/B-cells|Granulocytes|B-cells|T/NK cells|
|---          |---        |---      |---    |---        |---   |---          |---      |---        |---   |---       |---      |---                        |---             |---|---|---|---|---|---|
|PDGFRA =CD140A|CLDN5      |VTN      |ACTA2  |DCN        |MBP   |CSPG4 =NG2    |PIFO     |RBFOX3 =NEUN|DCX   |AQP4      |AIF1     |PTPRC =CD45                 |CD14            |TREM2|CCR2|CD74|CD16/32|CD19|CD4|
|MCAM =CD146   |PECAM1 =CD31|PDGFRB   |MYOCD  |COL6A1     |ENPP2 |PDGFRA =CD140A|FOXJ1    |TUBB3      ||          |         |                           |ITGB2 =CD18 =CD11B||||ITGB2 =CD18 =CD11B||CD8A|
|FOXF2        |           |         |       |COL3A1     |      |             |DYNLRB2  |           ||          |         |                           |CD86            ||||CD15||CD8B|
|             |           |         |       |           |      |             |MEIG1    |           ||          |         |                           |ADGRE1 =F4/80    ||||||IL2RB|
||||||||||||||||||||IFNG|

In [ ]:
# plot marker genes
plt.rcParams['figure.figsize'] = [8, 6] # set plot sizes
marker_genes = ["Pdgfra", "Mcam","Foxf2", "Pecam1", "Cldn5","Vtn", "Pdgfrb","Acta2", "Myocd","Dcn", "Col6a1", "Mbp","Enpp2","Cspg4","Pifo","Foxj1","Dynlrb2","Meig1","Rbfox3","Tubb3","Dcx","Aqp4", "Aif1", "Ptprc", "Ccr2","Adgre1","Itgb2","Cd14","Cd86","Trem2","Vcan","Cd4","Cd19", "Cd8a","Il2rb","Cd244", "Cd74","Cd68","Ifng","Ptgdr2","Ccr3"]
marker_genes=[x for x in marker_genes if x in list(adata.var_names)] # remove those not in adata.var_names
sc.pl.umap(adata, ncols=3, color=marker_genes, size=3)

In [ ]:
# Run DE test for annotation (Wilcoxon)
sc.tl.rank_genes_groups(adata, 'cell_type', method='wilcoxon', key_added='wilcoxon')
sc.pl.rank_genes_groups(adata, n_genes=20, sharey=False, ncols=3, fontsize=13, key='wilcoxon')

In [ ]:
# Dotplot
sc.tl.dendrogram(adata, groupby="cell_type")
sc.pl.rank_genes_groups_dotplot(adata, n_genes=6, key="wilcoxon", groupby="cell_type");

Note: No re-annotation required

# DE analysis

In [ ]:
sc.tl.rank_genes_groups(adata, 'cell_type', method='wilcoxon', key_added = "dea")

In [ ]:
sc.pl.rank_genes_groups(adata, n_genes=20, sharey=False, key = "dea", fontsize = 10)

In [ ]:
sc.get.rank_genes_groups_df(adata, key = "dea", group = "Oligos")[0:15]

In [ ]:
sc.get.rank_genes_groups_df(adata, key = "dea", group = "Neurons")[0:15]["names"]

In [ ]:
sc.pl.rank_genes_groups_dotplot(adata, n_genes=6, key="dea", groupby="cell_type")

In [ ]:
sc.pl.dotplot(adata, var_names=["Fcrls","Hexb","P2ry12","Ptprc","Mertk","Bsg"], groupby="cell_type")

In [ ]:
sc.pl.dotplot(adata, var_names=["Fcrls","Hexb","P2ry12","Ptprc","Mertk","Mrc1","Tmem119","Fos","Junb"], groupby="Taxonomy_group")

# Focus on: Foxf2

In [ ]:
adata.obs["clusters"] = adata.obs['cell_type']

In [ ]:
gene="Foxf2"

In [ ]:
with rc_context({'figure.figsize': (7,7)}):
    sc.pl.umap(adata, color=['clusters',gene], legend_loc='on data', title='', legend_fontweight='normal', legend_fontoutline=2, legend_fontsize=10, size=4)

## Plot

In [ ]:
sc.pl.matrixplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
utils.summarize_gene_expression(adata = adata, gene = gene, groupby = "clusters", 
                          study_name = dataset_name, organism = organism,
                          export = True, output_dir = os.path.join(main_dir, "P06_Foxf2_per_celltype", "Foxf2_summarized")
                         )

# Focus on: Other genes

In [ ]:
target_genes = ["Foxo1", "Tek", "Nos3", "Htra1", "Egfl8", "Flt1", "Kdr", "Nrp1", "Nrp2", "Efnb2", "Itgb1", "Itga6", "Angpt2", "Cdh5", "Cldn5", "Ocln", "Ctnnb1"]

In [ ]:
other_genes_results = {
    gene: utils.summarize_gene_expression(adata, gene, study_name = dataset_name, organism = organism, groupby = "clusters",
                                    output_dir=os.path.join(main_dir, "P06_Foxf2_per_celltype", "Other_genes_summarized"), export=True
                                   ) for gene in target_genes
}

In [ ]:
# some plots

In [ ]:
sc.pl.matrixplot(adata, [target_genes[0]], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [target_genes[0]], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.matrixplot(adata, [target_genes[1]], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [target_genes[1]], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

# Correlate gene expression (Foxf2 and Foxo1)

Using MAGIC denoising

In [ ]:
import magic
import scprep

In [ ]:
#sc.pp.scale(ad_merged)

## ECs

In [ ]:
gg = ["Foxf2","Foxo1","Nos3"]

In [ ]:
adata_EC = adata[adata.obs.clusters == "Endothelial cells"]

In [ ]:
adata_EC

In [ ]:
adata_EC.layers

In [ ]:
#matrix = pd.DataFrame(adata_EC.X) # not compatible with sparse 
matrix = adata_EC.X
matrix.columns = adata_EC.var.index.tolist()

In [ ]:
cutoff_var = None

In [ ]:
scprep.plot.plot_library_size(matrix, cutoff=cutoff_var)

In [ ]:
# filter lowly expressed genes and cells with a small library size
#matrix = scprep.filter.filter_library_size(matrix, cutoff=cutoff_var)
#matrix.head()

Note: Skipped normalization as data is already log-normalized

In [ ]:
adata_EC.layers

### Creating the MAGIC operator
If you don't specify parameters, MAGIC creates an operator with the following default values: knn=5, knn_max = 3 * knn, decay=1, t=3.

In [ ]:
magic_op = magic.MAGIC()

### Running MAGIC with gene selection
The magic_op.fit_transform function takes the normalized data and an array of selected genes as its arguments. If no genes are provided, MAGIC will return a matrix of all genes. The same can be achieved by substituting the array of gene names with genes='all_genes'.

In [ ]:
%%time
emt_magic = magic_op.fit_transform(adata_EC, genes=['Foxf2', 'Foxo1', 'Nos3'])

### Visualizing gene-gene relationships

We can see gene-gene relationships much more clearly after applying MAGIC. Note that the change in absolute values of gene expression is not meaningful - the relative difference is all that matters.

In [ ]:
np.corrcoef(emt_magic[:,['Foxf2','Foxo1']].X, rowvar = False)[0][1]

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16, 6))
scprep.plot.scatter(x=adata_EC[:,'Foxf2'].X.todense(), y=adata_EC[:,'Foxo1'].X.todense(), c=adata_EC[:,'Nos3'].X.todense(), ax = ax1,
                    xlabel='Foxf2', ylabel='Foxo1', legend_title="Nos3", title='Before MAGIC')
scprep.plot.scatter(x=emt_magic[:,'Foxf2'].X, y=emt_magic[:,'Foxo1'].X, c=emt_magic[:,'Nos3'].X, ax=ax2,
                    xlabel='Foxf2', ylabel='Foxo1', legend_title="Nos3", title='After MAGIC')
plt.axline((0,0), slope=1, color="black", alpha=0.3, linestyle="--")
plt.tight_layout()
plt.show()

## PCs

In [ ]:
gg = ["Foxf2","Foxo1","Nos3"]

In [ ]:
adata_EC = adata[adata.obs.clusters == "Pericytes"]

In [ ]:
adata_EC

In [ ]:
adata_EC.layers

In [ ]:
#matrix = pd.DataFrame(adata_EC.X) # not compatible with sparse 
matrix = adata_EC.X
matrix.columns = adata_EC.var.index.tolist()

In [ ]:
cutoff_var = None

In [ ]:
scprep.plot.plot_library_size(matrix, cutoff=cutoff_var)

In [ ]:
# filter lowly expressed genes and cells with a small library size
#matrix = scprep.filter.filter_library_size(matrix, cutoff=cutoff_var)
#matrix.head()

Note: Skipped normalization as data is already log-normalized

In [ ]:
adata_EC.layers

### Creating the MAGIC operator
If you don't specify parameters, MAGIC creates an operator with the following default values: knn=5, knn_max = 3 * knn, decay=1, t=3.

In [ ]:
magic_op = magic.MAGIC()

### Running MAGIC with gene selection
The magic_op.fit_transform function takes the normalized data and an array of selected genes as its arguments. If no genes are provided, MAGIC will return a matrix of all genes. The same can be achieved by substituting the array of gene names with genes='all_genes'.

In [ ]:
%%time
emt_magic = magic_op.fit_transform(adata_EC, genes=['Foxf2', 'Foxo1', 'Nos3'])

### Visualizing gene-gene relationships

We can see gene-gene relationships much more clearly after applying MAGIC. Note that the change in absolute values of gene expression is not meaningful - the relative difference is all that matters.

In [ ]:
np.corrcoef(emt_magic[:,['Foxf2','Foxo1']].X, rowvar = False)[0][1]

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16, 6))
scprep.plot.scatter(x=adata_EC[:,'Foxf2'].X.todense(), y=adata_EC[:,'Foxo1'].X.todense(), c=adata_EC[:,'Nos3'].X.todense(), ax = ax1,
                    xlabel='Foxf2', ylabel='Foxo1', legend_title="Nos3", title='Before MAGIC')
scprep.plot.scatter(x=emt_magic[:,'Foxf2'].X, y=emt_magic[:,'Foxo1'].X, c=emt_magic[:,'Nos3'].X, ax=ax2,
                    xlabel='Foxf2', ylabel='Foxo1', legend_title="Nos3", title='After MAGIC')
plt.axline((0,0), slope=1, color="black", alpha=0.3, linestyle="--")
plt.tight_layout()
plt.show()

## SMCs

In [ ]:
gg = ["Foxf2","Foxo1","Nos3"]

In [ ]:
adata_EC = adata[adata.obs.clusters == "SMCs"]

In [ ]:
adata_EC

In [ ]:
adata_EC.layers

In [ ]:
#matrix = pd.DataFrame(adata_EC.X) # not compatible with sparse 
matrix = adata_EC.X
matrix.columns = adata_EC.var.index.tolist()

In [ ]:
cutoff_var = None

In [ ]:
scprep.plot.plot_library_size(matrix, cutoff=cutoff_var)

In [ ]:
# filter lowly expressed genes and cells with a small library size
#matrix = scprep.filter.filter_library_size(matrix, cutoff=cutoff_var)
#matrix.head()

Note: Skipped normalization as data is already log-normalized

In [ ]:
adata_EC.layers

### Creating the MAGIC operator
If you don't specify parameters, MAGIC creates an operator with the following default values: knn=5, knn_max = 3 * knn, decay=1, t=3.

In [ ]:
magic_op = magic.MAGIC()

### Running MAGIC with gene selection
The magic_op.fit_transform function takes the normalized data and an array of selected genes as its arguments. If no genes are provided, MAGIC will return a matrix of all genes. The same can be achieved by substituting the array of gene names with genes='all_genes'.

In [ ]:
%%time
emt_magic = magic_op.fit_transform(adata_EC, genes=['Foxf2', 'Foxo1', 'Nos3'])

### Visualizing gene-gene relationships

We can see gene-gene relationships much more clearly after applying MAGIC. Note that the change in absolute values of gene expression is not meaningful - the relative difference is all that matters.

In [ ]:
np.corrcoef(emt_magic[:,['Foxf2','Foxo1']].X, rowvar = False)[0][1]

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16, 6))
scprep.plot.scatter(x=adata_EC[:,'Foxf2'].X.todense(), y=adata_EC[:,'Foxo1'].X.todense(), c=adata_EC[:,'Nos3'].X.todense(), ax = ax1,
                    xlabel='Foxf2', ylabel='Foxo1', legend_title="Nos3", title='Before MAGIC')
scprep.plot.scatter(x=emt_magic[:,'Foxf2'].X, y=emt_magic[:,'Foxo1'].X, c=emt_magic[:,'Nos3'].X, ax=ax2,
                    xlabel='Foxf2', ylabel='Foxo1', legend_title="Nos3", title='After MAGIC')
plt.axline((0,0), slope=1, color="black", alpha=0.3, linestyle="--")
plt.tight_layout()
plt.show()

## All cell types

In [ ]:
adata

In [ ]:
#matrix = pd.DataFrame(adata.X) # not compatible with sparse 
matrix = adata.X
#matrix.columns = ad_merged.var.index.tolist()

In [ ]:
cutoff_var = 700

In [ ]:
scprep.plot.plot_library_size(matrix, cutoff=cutoff_var)

In [ ]:
# filter lowly expressed genes and cells with a small library size
#matrix = scprep.filter.filter_library_size(matrix, cutoff=cutoff_var)
#matrix.head()

Note: Skipped normalization as data is already log-normalized

In [ ]:
adata.layers

### Creating the MAGIC operator
If you don't specify parameters, MAGIC creates an operator with the following default values: knn=5, knn_max = 3 * knn, decay=1, t=3.

In [ ]:
magic_op = magic.MAGIC()

### Running MAGIC with gene selection
The magic_op.fit_transform function takes the normalized data and an array of selected genes as its arguments. If no genes are provided, MAGIC will return a matrix of all genes. The same can be achieved by substituting the array of gene names with genes='all_genes'.

In [ ]:
%%time
emt_magic = magic_op.fit_transform(adata, genes=['Foxf2', 'Foxo1', 'Nos3'])

### Visualizing gene-gene relationships

We can see gene-gene relationships much more clearly after applying MAGIC. Note that the change in absolute values of gene expression is not meaningful - the relative difference is all that matters.

In [ ]:
np.corrcoef(emt_magic[:,['Foxf2','Foxo1']].X, rowvar = False)[0][1]

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(13, 6))

scprep.plot.scatter(x=adata[:,'Foxf2'].X.todense(), y=adata[:,'Foxo1'].X.todense(), c=adata[:,'Nos3'].X.todense(), ax = ax1,
                    xlabel='Foxf2', ylabel='Foxo1', legend_title="Nos3", title='Before MAGIC')

scprep.plot.scatter(x=emt_magic[:,'Foxf2'].X, y=emt_magic[:,'Foxo1'].X, c=emt_magic[:,'Nos3'].X, ax=ax2,
                    xlabel='Foxf2', ylabel='Foxo1', title='After MAGIC')
plt.axline((0,0), slope=1, color="black", alpha=0.3, linestyle="--")
plt.tight_layout()
plt.show()

# Save

In [ ]:
adata

In [ ]:
name='zeisel_normalised_logarithmised_annotated'

In [ ]:
adata.write(main_dir+"\P06_Foxf2_per_celltype\\scRNAseq\\" + date.today().strftime("%Y%m%d")+'_'+name+'.h5ad')

# Session Info

In [ ]:
sc.logging.print_versions()